In [2]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

Download the Fashion-MNIST dataset

In [3]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

Upload Fashion-MNIST data to S3

In [ ]:
prefix = 'keras-fashion-mnist'

training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

Train with Tensorflow on the notebook instance (aka 'local mode')

In [ ]:
!pygmentize mnist_keras_tf.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

Deploy

In [ ]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                  instance_type='ml.t3.medium',
                                  endpoint_name=tf_endpoint_name)      # $1.361/hour in eu-west-1

# tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                          instance_type='ml.c5.large',        # $0.134/hour in eu-west-1
#                          accelerator_type='ml.eia1.medium',  # + $0.140/hour in eu-west-1
#                          endpoint_name=tf_endpoint_name)     # = 80% discount!

Predict

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

Clean up

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)